# Setup

Ensure all dependencies are installed.


In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


Check if we have GPU support, and if not, warn the user.


In [2]:
import torch
import warnings

if torch.cuda.is_available():
    print("CUDA is available!")
else:
    warnings.warn("CUDA is not available. Running on CPU.")

CUDA is available!


Load our dataset...


In [ ]:
import tempfile
import os
import zipfile
import shutil

import utils.file

from data_sources import DataSource, ModelNet40
from pointnet.dataset import ModelNetDataset

data_source: DataSource = ModelNet40()
data_size = utils.file.get_download_size(data_source.url)

temp_dir = tempfile.gettempdir()
zip_path = os.path.join(temp_dir, data_source.name + ".zip")
dataset_path = os.path.join(temp_dir, data_source.name)

if os.path.exists(zip_path) and utils.file.checksum(zip_path) == data_source.checksum:
    print("Zip already downloaded.")
else:
    utils.file.download_file(data_source.url, zip_path)
    print("Zip downloaded.")

if os.path.exists(dataset_path):
    print("Conflicting data folder found. Deleting existing folder...")
    shutil.rmtree(dataset_path)
    print("Folder deleted.")

print("Extracting data...")
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(dataset_path)
    print("Data extracted.")

npoints = 2500

train_dataset = ModelNetDataset(root=dataset_path, npoints=npoints, split="train")

test_dataset = ModelNetDataset(
    root=dataset_path,
    split="test",
    npoints=npoints,
)

print(
    f"Training data: {len(train_dataset)} samples with {len(train_dataset.classes)} classes"
)
print(
    f"Test data size: {len(test_dataset)} samples with {len(test_dataset.classes)} classes"
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Zip already downloaded.
Conflicting data folder found. Deleting existing folder...
Folder deleted.
Extracting data...


# Training

Train a new classifier or load a new pre-trained one:


In [ ]:
from pointnet.train_classification import train_classification
from pointnet.model import PointNetCls

classifier_path = "./pointnet/classifier.pth"

if classifier_path and os.path.exists(classifier_path):
    print(f"Loading existing model from {classifier_path}...")
    classifier = PointNetCls(k=len(train_dataset.classes))
    classifier.load_state_dict(torch.load(classifier_path, weights_only=True))
    print("Classifier loaded successfully.")
else:
    classifier = train_classification(
        train_dataset=train_dataset,
        test_dataset=test_dataset,
        epochs=25,
        outf="./pointnet/cls",
    )

Loading existing model from ./pointnet/classifier.pth...
Classifier loaded successfully.


Run a simple classification to get a visualization of our classifier.


In [ ]:
import random
import torch
from utils.plot import plot_points

i = random.randint(0, len(test_dataset) - 1)

input_data = test_dataset[i][0].unsqueeze(0)
input_data = input_data.transpose(1, 2)

classifier.eval()

with torch.no_grad():
    output = classifier(input_data)


scores: torch.Tensor = output[0][0]
print("Scores:", scores.tolist())
print(
    f"Expected class: {test_dataset.classes[int(test_dataset[i][1])]} ({int(test_dataset[i][1])})",
)
print(
    f"Predicted class: {test_dataset.classes[scores.argmax().item()]} ({scores.argmax().item()})"
)

points = input_data[0].transpose(0, 1).numpy()
plot_points(points)

ValueError: Cannot take a larger sample than population when 'replace=False'

Train the segmenter:


In [ ]:
from pointnet.train_segmentation import train_segmentation

train_segmentation(
    train_dataset=train_dataset, test_dataset=test_dataset, epochs=25, class_choice=0
)

Random Seed:  9583


ValueError: Expected input batch_size (80000) to match target batch_size (32).